https://keras-team.github.io/keras-tuner/ <br/>
https://autokeras.com/tutorial/image_classification/ <br/>
무조건 코랩에서 하자. CPU 터진다

## data preparing

In [5]:
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, random_state = 777)

num_x_train = x_train.shape[0]
num_x_val = x_val.shape[0]
num_x_test = x_test.shape[0]

x_train = (x_train.reshape(-1, 28, 28, 1)) / 255
x_val = (x_val.reshape(-1, 28, 28, 1)) / 255
x_test = (x_test.reshape(-1, 28, 28, 1)) / 255

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

## Standard CNN
일반적인 cnn 구조에서 고려할 수 있는 하이퍼파라미터들
* conv2d layer : num of filters
* Dense layer : num of hidden units
* dropout ratio
* GAP or GMP ?
* Adam : learning

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam

inputs = Input(shape = (28, 28, 1))

x = Conv2D(32, (3, 3), activation = 'relu')(inputs)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = MaxPooling2D(strides = 2)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(30, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = x)

model.compile(optimizer = Adam(learning_rate = 0.001), 
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

## Define Keras Tuner model

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    inputs = Input(shape = (28, 28, 1))
    x = inputs

    for i in range(hp.Int('n_layers', 1, 3)):
        # 필터 개수 탐색
        x = Conv2D(
            filters = hp.Int('filters_' + str(i), 4, 64, step = 8, default = 16),
            kernel_size = (3, 3), activation = 'relu',
            padding = 'same')(x)
    x = MaxPooling2D(strides = 2)(x)

    # GAP? GMP?
    if hp.Choice('global_pooling', ['max', 'avg']) == 'avg':
        x = GlobalAveragePooling2D()(x)
    else:
        x = GlobalMaxPooling2D()(x)

    # 은닉층 노드 개수 탐색
    x = Dense(units = hp.Int('units', 
                          min_value = 16,
                          max_value = 128,
                          step = 16),
            activation = 'relu')(x)
    
    # 드롭아웃률 탐색
    x = Dropout(hp.Choice('dropout_rate', values = [0.2, 0.3, 0.5]))(x)
    x = Dense(10, activation = 'softmax')(x)

    model = Model(inputs = inputs, outputs = x)
    # 학습률 탐색
    model.compile(optimizer = Adam(hp.Choice('learning_rate', 
                                             values = [1e-3, 1e-4, 1e-5])),
                loss = 'categorical_crossentropy',
                metrics = ['acc'])

    return model

## Define tuner and Search param. space

In [8]:
from kerastuner.tuners import RandomSearch # or Hyperband

tuner = RandomSearch(build_model,                        # 탐색할 모델
                     objective='val_acc',                # 모니터링 지표
                     max_trials=5,                       # 실험 진행 수 (파라미터세팅)
                     executions_per_trial=3,             # 각 실험당 반복 수
                     directory='C:\\Users\\user\\study', # 결과 저장될 디렉터리
                     project_name='tuner_ex')

In [9]:
# parameter space
tuner.search_space_summary()

Search space summary
Default search space size: 6
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
filters_0 (Int)
{'default': 16, 'conditions': [], 'min_value': 4, 'max_value': 64, 'step': 8, 'sampling': None}
global_pooling (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': None}
dropout_rate (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.5], 'ordered': True}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05], 'ordered': True}


In [10]:
# search param space
tuner.search(x=x_train,y=y_train,
             epochs=10,
             validation_data=(x_val, y_val))

Trial 5 Complete [00h 30m 54s]
val_acc: 0.951370378335317

Best val_acc So Far: 0.951370378335317
Total elapsed time: 02h 47m 44s
INFO:tensorflow:Oracle triggered exit


In [11]:
# 실험 결과 요약
tuner.results_summary()

Results summary
Results in C:\Users\user\study\tuner_ex
Showing 10 best trials
Objective(name='val_acc', direction='max')
Trial summary
Hyperparameters:
n_layers: 3
filters_0: 20
global_pooling: max
units: 48
dropout_rate: 0.5
learning_rate: 0.001
filters_1: 12
filters_2: 20
Score: 0.951370378335317
Trial summary
Hyperparameters:
n_layers: 3
filters_0: 20
global_pooling: max
units: 128
dropout_rate: 0.3
learning_rate: 0.001
filters_1: 12
filters_2: 12
Score: 0.939981480439504
Trial summary
Hyperparameters:
n_layers: 2
filters_0: 44
global_pooling: max
units: 96
dropout_rate: 0.3
learning_rate: 1e-05
filters_1: 16
Score: 0.46174074212710065
Trial summary
Hyperparameters:
n_layers: 2
filters_0: 20
global_pooling: avg
units: 112
dropout_rate: 0.5
learning_rate: 0.0001
filters_1: 12
filters_2: 20
Score: 0.3933148185412089
Trial summary
Hyperparameters:
n_layers: 3
filters_0: 52
global_pooling: avg
units: 32
dropout_rate: 0.2
learning_rate: 1e-05
filters_1: 20
filters_2: 16
Score: 0.2569629

## best model

In [12]:
# 성능이 가장 좋았던 모델 호출
best_model = tuner.get_best_models()[0]
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 20)        200       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 12)        2172      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 20)        2180      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 20)        0         
_________________________________________________________________
global_max_pooling2d (Global (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 48)                1008  

In [13]:
# best 모델의 하이퍼파라미터 확인
best_hp = tuner.get_best_hyperparameters()[0].values
best_hp

{'n_layers': 3,
 'filters_0': 20,
 'global_pooling': 'max',
 'units': 48,
 'dropout_rate': 0.5,
 'learning_rate': 0.001,
 'filters_1': 12,
 'filters_2': 20}

> ### keras tuner 사용절차
>1. hp를 parameter로 받는 model build함수 정의
2. (1)의 함수 정의 과정에서 하이퍼파라미터 튜닝이 필요한 부분에 hp.Int(), hp.Choice()등 초모수 성격에 따라 적절히 범위/범주를 설정하여 api에 맞게 옵션을 걸어준다.
3. 사용할 searching 알고리즘(RS, Bayes, HB 등)을 정하여, 실험 및 반복수 등을 설정
4. 정의한 searching 알고리즘을 task dataset에 적용시킨다.
5. 결과를 확인한다.

## HyperResNet

In [ ]:
from kerastuner.tuners import Hyperband
from kerastuner.applications import HyperResNet

hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

tuner = Hyperband(hypermodel,
                  objective = 'val_accuracy',
                  max_epochs = 10,
                  directory='C:\\Users\\user\\study',
                  project_name='tuner_resnet')

In [ ]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_val, y_val))